# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. Make sure Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [121]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/tfkeras' Feel free to use this Image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

In [122]:
resource_group = "fbaksrg" # Feel free to modify these
aks_name = "fbAKSCluster"
location = "eastus"

image_name = "fboylu/tfkeras" 
selected_subscription = "'Team Danielle Internal'"

## Azure acocunt login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [129]:
!az login -o table

In [4]:
!az account set --subscription $selected_subscription

In [130]:
!az account show

In [5]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [131]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [10]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


In [9]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

SSH key files '/home/fboylu/.ssh/id_rsa' and '/home/fboylu/.ssh/id_rsa.pub' have been generated under ~/.ssh to allow SSH access to the VM. If using machines without permanent storage like Azure Cloud Shell without an attached file share, back up your keys to a safe location
Waiting for AAD role to propagate[################################    ]  90.0000%%Could not create a service principal with the right permissions. Are you an Owner on this project?
^CK - Running ...
Long-running operation wait cancelled.  


### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [12]:
!sudo az aks install-cli

## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [15]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSCluster" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [132]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     11d       v1.7.7


Let's check the pods on our cluster.

In [124]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   heapster-2574232661-9rnfw               2/2       Running   0          11d
kube-system   kube-dns-v20-2253765213-nc3lc           3/3       Running   0          11d
kube-system   kube-dns-v20-2253765213-tlrxc           3/3       Running   0          11d
kube-system   kube-proxy-g4gs2                        1/1       Running   0          11d
kube-system   kube-svc-redirect-57f3x                 1/1       Running   4          11d
kube-system   kubernetes-dashboard-2898242510-5f9m0   1/1       Running   0          11d
kube-system   tunnelfront-2759728499-dgvc5            1/1       Running   0          11d


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [127]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/tfkeras",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "name": "bin",
                              "mountPath":"/usr/local/nvidia/bin" 
                          },
                          {
                              "name": "lib",
                              "mountPath":"/usr/local/nvidia/lib64" 
                          },
                          {
                              "name": "libcuda",
                              "mountPath":"/usr/lib/x86_64-linux-gnu/libcuda.so.1" 
                          },
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "bin",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384/bin"
                      },
                  },
                  {
                      "name": "lib",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384"
                      },
                  },
                  {
                      "name": "libcuda",
                      "hostPath":{
                          "path":"/usr/lib/x86_64-linux-gnu/libcuda.so.1"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [133]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4,sort_keys=True)
        outfile.write('\n\n')

In [134]:
write_json_to_file(app_template, 'az-dl.json')

In [135]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [136]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "fboylu/tfkeras",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                   

Next, we will use kubectl create command to deploy our application.

In [137]:
!kubectl create -f az-dl.json

deployment "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [139]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS              RESTARTS   AGE
default       azure-dl-2408918338-m0s9t               0/1       ContainerCreating   0          40s
kube-system   heapster-2574232661-9rnfw               2/2       Running             0          11d
kube-system   kube-dns-v20-2253765213-nc3lc           3/3       Running             0          11d
kube-system   kube-dns-v20-2253765213-tlrxc           3/3       Running             0          11d
kube-system   kube-proxy-g4gs2                        1/1       Running             0          11d
kube-system   kube-svc-redirect-57f3x                 1/1       Running             4          11d
kube-system   kubernetes-dashboard-2898242510-5f9m0   1/1       Running             0          11d
kube-system   tunnelfront-2759728499-dgvc5            1/1       Running             0          11d


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [144]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                             SOURCE                              MESSAGE
56s         11d          17208     aks-nodepool1-28016997-0.151e0b00f2c2fa81    Node                                     Warning   FailedNodeAllocatableEnforcement   kubelet, aks-nodepool1-28016997-0   Failed to update Node Allocatable Limits "": failed to set supported cgroup subsystems for cgroup : Failed to set config for supported subsystems : failed to write 59076296704 to memory.limit_in_bytes: write /var/lib/docker/overlay2/b13d017f80b0afbea6242f387627df263ea463c385c0655defa6221c49714e66/merged/sys/fs/cgroup/memory/memory.limit_in_bytes: invalid argument
6m          6m           1         azure-dl-2408918338-m0s9t.1521b5ae1eeba886   Pod                                      Normal    Scheduled                          default-scheduler                   Successfully assigne

Check the logs for the application pod.

In [152]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [154]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-04-02 19:40:18,693 CRIT Supervisor running as root (no user in config file)
2018-04-02 19:40:18,695 INFO supervisord started with pid 8
2018-04-02 19:40:19,697 INFO spawned: 'program_exit' with pid 16
2018-04-02 19:40:19,699 INFO spawned: 'nginx' with pid 17
2018-04-02 19:40:19,700 INFO spawned: 'gunicorn' with pid 18
2018-04-02 19:40:20,731 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-04-02 19:40:21.168814: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-04-02 19:40:25,173 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-04-02 19:40:31.043704: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: d333:00:00.0
totalMemory: 11.17GiB freeMe

In [155]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           12m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [157]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.214.10   52.168.106.16   80:32210/TCP   13m


# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [120]:
!kubectl delete -f az-dl.json

deployment "azure-dl" deleted
service "azure-dl" deleted


In [ ]:
!az aks delete -n $aks_name -g $resource_group -y

In [ ]:
!az group delete --name $resource_group -y